In [32]:
import tensorflow as tf
from keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout
import numpy as np
import pandas as pd

In [33]:
train_dir = 'archive/Dataset/Train'
validation_dir = 'archive/Dataset/Validation'
test_dir = 'archive/Dataset/Test'

In [7]:
from PIL import Image 
import os
train_dir = 'archive/Dataset/Train'
folder_name = 'Fake'
image_name = 'fake_1.jpg'
folder_path = os.path.join(train_dir, folder_name)
image_path = os.path.join(folder_path, image_name)
if os.path.exists(folder_path):
    if os.path.exists(image_path):
        image = Image.open(image_path)
        image.show()
    else:
        print(f"The image {image_name} does not exist in the folder {folder_name}.")
else:
    print(f"The folder {folder_name} does not exist in the train directory.")

In [8]:
def get_pixel(img, x1, y1, x, y):
    new_value = 0
    try:
        if img[x1][y1] >= img[x][y]:
            new_value = 1
    except IndexError:
        pass
    return new_value

def cs_lbp_calculated_pixel(img, x, y):
    val_ar = []
    
    val_ar.append(get_pixel(img, x, y+1, x, y-1))
    val_ar.append(get_pixel(img, x+1, y+1, x-1, y - 1))
    val_ar.append(get_pixel(img, x+1, y, x-1, y))
    val_ar.append(get_pixel(img, x+1, y-1, x - 1, y + 1))

    power_val = [1, 2, 4, 8]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val

In [9]:
def apply_cs_lbp_clahe(image, output_directory, label, clip_limit=2.0, grid_size=(8, 8)):
    img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    img_clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size).apply(img_gray)
    height, width = img_clahe.shape
    img_cs_lbp = np.zeros((height, width), np.uint16)

    for i in range(0, height):
        for j in range(0, width):
            img_cs_lbp[i, j] = cs_lbp_calculated_pixel(img_clahe, i, j)

    # Save the CS-LBP image
    cv2.imwrite(os.path.join(output_directory, label), img_cs_lbp)
    
    #plt.imshow(img_cs_lbp, cmap="gray")
    #plt.title("CS-LBP Image")
    #plt.show()

In [11]:
import cv2
import os

# Define CLAHE parameters
clip_limit = 2.0
grid_size = (8, 8)

# Function to apply CS-LBP and CLAHE transformations
def apply_cs_lbp_clahe(img, output_dir, filename):
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply CLAHE
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=grid_size)
    clahe_img = clahe.apply(gray)
    
    # Apply CS-LBP
    # Assuming CS-LBP logic is implemented elsewhere
    
    # Save the processed image
    output_path = os.path.join(output_dir, filename)
    cv2.imwrite(output_path, clahe_img)
    
# Function to extract face ROI from an image
def extract_face(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Assuming face ROI extraction logic is implemented elsewhere
    
    return img # Placeholder return for demonstration

# Define the function to process each dataset
def process_dataset(real_dir, fake_dir, output_parent_dir):
    for label, directory in zip(['Real', 'Fake'], [real_dir, fake_dir]):
        output_dir = os.path.join(output_parent_dir, label)
        os.makedirs(output_dir, exist_ok=True)
        
        # Process each image in the directory
        for image_file in os.listdir(directory):
            image_path = os.path.join(directory, image_file)
            face_roi = extract_face(image_path)
            if face_roi is not None:
                apply_cs_lbp_clahe(face_roi, output_dir, image_file)

# Set the paths for each dataset
train_real_dir = 'archive/Dataset/Train/Real'
train_fake_dir = 'archive/Dataset/Train/Fake'
test_real_dir = 'archive/Dataset/Test/Real'
test_fake_dir = 'archive/Dataset/Test/Fake'
validation_real_dir = 'archive/Dataset/Test/Fake'  # Should this be 'archive (2)/Dataset/Validation/Real'?
validation_fake_dir = 'archive/Dataset/Validation/Fake'
output_dir = 'archive/Dataset/output'


# Process each dataset
process_dataset(train_real_dir, train_fake_dir, os.path.join(output_dir, 'training'))
process_dataset(test_real_dir, test_fake_dir, os.path.join(output_dir, 'testing'))
process_dataset(validation_real_dir, validation_fake_dir, os.path.join(output_dir, 'validation'))

print("CS-LBP transformation with CLAHE completed for all images in the train, validation, and test datasets.")

CS-LBP transformation with CLAHE completed for all images in the train, validation, and test datasets.


In [21]:
train_dir = 'archive/Dataset/output/training'
validation_dir = 'archive/Dataset/output/training'
test_dir = 'archive/Dataset/output/training'

In [22]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

In [23]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',  # Assuming a binary classification task (fake vs real)
    classes=['Fake','Real']  # Specify the class names
    
)

Found 140002 images belonging to 2 classes.


In [24]:
validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    classes=['Fake', 'Real']
)

Found 140002 images belonging to 2 classes.


In [25]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    classes=['Fake', 'Real']
)

Found 140002 images belonging to 2 classes.


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories for training, validation, and test data
train_dir = 'archive/Dataset/Train'
validation_dir = 'archive/Dataset/Validation'
test_dir = 'archive/Dataset/Test'

# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Create ImageDataGenerator instances for training, validation, and test data
train_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')

validation_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
validation_generator = validation_datagen.flow_from_directory(validation_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')

test_datagen = ImageDataGenerator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)
test_generator = test_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')

# Load MobileNetV2 model pre-trained on ImageNet dataset
base_model = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

# Freeze the pre-trained layers
base_model.trainable = False

# Add custom classification head
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output_layer2 = tf.keras.layers.Dense(1, activation='sigmoid')(global_average_layer)

# Create the model
model = tf.keras.models.Model(inputs=base_model.input, outputs=output_layer2)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=7, validation_data=validation_generator)

# Evaluate the model on test data

test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

Found 140002 images belonging to 2 classes.
Found 39428 images belonging to 2 classes.
Found 10905 images belonging to 2 classes.
Epoch 1/7


E:\Anaconda\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1486s 338ms/step - accuracy: 0.7594 - loss: 0.4830 - val_accuracy: 0.7867 - val_loss: 0.4500
Epoch 2/7
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1418s 324ms/step - accuracy: 0.7961 - loss: 0.4274 - val_accuracy: 0.7833 - val_loss: 0.4557
Epoch 3/7
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1423s 325ms/step - accuracy: 0.7995 - loss: 0.4220 - val_accuracy: 0.7906 - val_loss: 0.4415
Epoch 4/7
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1426s 326ms/step - accuracy: 0.8002 - loss: 0.4226 - val_accuracy: 0.7734 - val_loss: 0.4749
Epoch 5/7
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1426s 326ms/step - accuracy: 0.8009 - loss: 0.4195 - val_accuracy: 0.7905 - val_loss: 0.4425
Epoch 6/7
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1439s 329ms/step - accuracy: 0.8027 - loss: 0.4177 - val_accuracy: 0.7816 - val_loss: 0.4642
Epoch 7/7
4376/4376 ━━━━━━━━━━━━━━━━━━━━ 1449s 331ms/step - accuracy: 0.8011 - loss: 0.4188 - val_accuracy: 0.7947 - val_loss: 0.4355
341/341 ━━━━━━━━━━━━━━━━━━━━ 94s 276ms/step - accuracy: 0.6969 - loss: 0

In [3]:
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy:', test_acc)

341/341 ━━━━━━━━━━━━━━━━━━━━ 87s 255ms/step - accuracy: 0.6960 - loss: 0.6355
Test accuracy: 0.6953691244125366


In [4]:
validation_loss, validation_acc = model.evaluate(validation_generator)
print('Test accuracy:', validation_acc)

1233/1233 ━━━━━━━━━━━━━━━━━━━━ 316s 256ms/step - accuracy: 0.7936 - loss: 0.4392
Test accuracy: 0.7947397828102112


In [29]:
model.save('my_model2.h5')

In [6]:
model.save('my_model2.keras')